# 1. 파일 열어서 title/script 얻기 (따로 저장도 했음)

In [ ]:
import csv
import re

# CSV 파일 열기 및 읽기
csv.field_size_limit(100000000)
with open('/content/movie_scripts.csv', 'r') as f:
    rdr = csv.reader(f)

    scriptlist = []
    titlelist = []

    # 영화 제목과 대본 리스트 생성
    for line in rdr:
        if len(line) < 2:
            continue
        titlelist.append(line[0])

        # 대본 포맷팅
        formatted_script = line[1]

        # 탭을 스페이스로 변경하고 캐리지 리턴 삭제
        formatted_script = re.sub("\t", " ", formatted_script)
        formatted_script = re.sub("\r", "", formatted_script)

        # 여러 개의 스페이스를 하나로 변경
        formatted_script = re.sub("  +", " ", formatted_script)

        # 모든 줄 바꿈(\n)을 스페이스로 변경
        formatted_script = formatted_script.replace("\n", " ")

        # 양 끝의 공백 제거
        formatted_script = formatted_script.strip()

        # 포맷팅된 대본 리스트에 추가
        scriptlist.append(formatted_script)

# # 새로운 CSV 파일에 제목과 대본 저장
# with open('/content/new_movie_scripts.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     # 헤더 작성
#     writer.writerow(['title', 'script'])
#     # 영화 제목과 대본을 한 줄씩 작성
#     for title, script in zip(titlelist, scriptlist):
#         writer.writerow([title, script])

In [ ]:
print(len(scriptlist), len(titlelist))

1212 1212


In [ ]:
scriptlist = scriptlist[1:]
titlelist = titlelist[1:] # col 제외

In [ ]:
print(len(scriptlist), len(titlelist))

1211 1211


In [ ]:
print(scriptlist[0])

12 AND HOLDING      Written by    Anthony S Cipriano          04.06.04        FADE IN:    EXT. NEIGHBORHOOD STREET - MORNING    TWIN BOYS, RUDY AND JACOB CARGES (12), ride their bikes  through a suburban neighborhood.    Rudy, the more athletic of the two, rides at a breakneck  pace. Jacob rides slowly due to a HOCKEY MASK that he wears  over his face. It's making it difficult for him to see. The  boys turn down a DIRT PATH and ride deep into some WOODS.    INT. WOODS - CONTINUOUS    Rudy and Jacob ride to the edge of a CLEARING and stop.  Across from the clearing is a large OAK TREE, which has a  TREE HOUSE perched high up in it's branches. The boys  cautiously look around and whisper.    JACOB  You see `em?    RUDY  No. But that doesn't mean they're  not here.    After a beat, Rudy gets off his bike and starts walking  towards the tree house. Jacob stays behind, eyeing their  safety.    RUDY (CONT'D) (cont'd)  Jacob, come on. It's cool.    Suddenly, a ROCK comes careening from off sc

In [ ]:
# 새로운 CSV 파일에 제목과 대본 저장
with open('/content/new_movie_scripts.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    # 헤더 작성
    writer.writerow(['title', 'script'])
    # 영화 제목과 대본을 한 줄씩 작성
    for title, script in zip(titlelist, scriptlist):
        writer.writerow([title, script])

# 2. Preprocessing

In [ ]:
# Load CSV file
input_csv_path = "/content/new_movie_scripts.csv"
output_csv_path = "/content/scripts_preprocessed.csv"
df = pd.read_csv(input_csv_path)

# Remove rows with NaN values in the second column
#df = df.dropna(subset=[df.columns[1]])
df = df.dropna()

# Remove rows where the length of the text in the second column is less than 20 before preprocessing
df = df[df.iloc[:, 1].str.len() >= 20]

In [ ]:
len(df) # 1211이었음 원래

1136

In [ ]:
import pandas as pd
import spacy
import nltk
import re
from nltk.corpus import stopwords
from autocorrect import Speller

# Load spaCy model for Named Entity Recognition
nlp = spacy.load("en_core_web_sm")

# Download NLTK stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Initialize the Speller from autocorrect
spell = Speller("en")

i = 0

# Function to preprocess text using regex, spaCy for NER, and autocorrect for spelling correction
def preprocess_text(text):
    global i

    if not isinstance(text, str):
        return ""

    # Correct spelling mistakes
    # text = spell(text)

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove specified named entities
    doc = nlp(text)
    tokens = []

    for token in doc:
        if token.ent_type_ not in ['PERSON', 'GPE', 'LOC', 'ORG', 'PRODUCT', 'DATE', 'TIME', 'PERCENT', 'QUANTITY', 'ORDINAL', 'CARDINAL', 'FAC']:
            tokens.append(token.text)

    # Join tokens to form the cleaned text
    cleaned_text = " ".join(tokens)

    # Remove stop words
    cleaned_tokens = [word for word in cleaned_text.split() if word.lower() not in stop_words]

    # Join tokens to form the final cleaned text
    final_cleaned_text = " ".join(cleaned_tokens)

    # Correct spelling mistakes one more time
    # final_cleaned_text = spell(final_cleaned_text)

    # Make all texts to lower case
    final_cleaned_text = final_cleaned_text.lower()

    print(f"{i}번째 완료")
    #i += 1
    return final_cleaned_text

# Preprocess the movie scripts in the second column
df['script'] = df.iloc[:, 1].apply(preprocess_text)

# Remove rows where the 'script' length is less than 20 after preprocessing
df = df[df['script'].str.len() >= 20]

# Save the preprocessed scripts to a new CSV file
df.to_csv(output_csv_path, index=False)

print("Preprocessing complete. Cleaned scripts saved to:", output_csv_path)

In [ ]:
print(len(df)) # 1211 >> 1136으로 1차로 줄였던 것
print(df['script'][0])

1123
holding written fade ext neighborhood street morning twin boys rudy jacob carges ride bikes suburban neighborhood athletic rides breakneck pace rides slowly due mask wears face making difficult see boys turn dirt path ride deep woods int woods ride edge stop across clearing large oak tree tree house perched high branches boys cautiously look around whisper nt mean beat gets bike starts walking towards tree house stays behind eyeing safety rudy contd contd come cool suddenly rock comes careening screen hits head knocks ground darts oak tree hail storm rock debris come flying rises stream blood pours front mask quickly runs tree boys trailer park punks come running woods rocks hand climb tree using makeshift rungs nailed trunk floor tree house removes key chain around neck unlocks climbs inside pulls house looks back sees running crosses door bucket liquid jacob hell rudy piss dumps piss onto time drenched piss jump tree screaming try shake urine spits taste mouth angrily calls boys

In [ ]:
import pandas as pd
df = pd.read_csv("/content/scripts_preprocessed.csv")

In [ ]:
df.head()

,title,script
0,12 and Holding,holding written fade ext neighborhood street m...
1,187,revised shooting draft educational purposes re...
2,1492: Conquest of Paradise,b bif window top toplocationhreflocationhref b...
3,17 Again,written october ext fitch senior high school d...
4,15 Minutes,b bif window top toplocationhreflocationhref b...


In [ ]:
print(df['script'][0]) # looks not that bad, but some typo

holding written fade ext neighborhood street morning twin boys rudy jacob carges ride bikes suburban neighborhood athletic rides breakneck pace rides slowly due mask wears face making difficult see boys turn dirt path ride deep woods int woods ride edge stop across clearing large oak tree tree house perched high branches boys cautiously look around whisper nt mean beat gets bike starts walking towards tree house stays behind eyeing safety rudy contd contd come cool suddenly rock comes careening screen hits head knocks ground darts oak tree hail storm rock debris come flying rises stream blood pours front mask quickly runs tree boys trailer park punks come running woods rocks hand climb tree using makeshift rungs nailed trunk floor tree house removes key chain around neck unlocks climbs inside pulls house looks back sees running crosses door bucket liquid jacob hell rudy piss dumps piss onto time drenched piss jump tree screaming try shake urine spits taste mouth angrily calls boys kenn

## label 칼럼 추가

In [ ]:
import pandas as pd

path = "/content/evaluation.csv"  # labelled data
df = pd.read_csv(path)

# Prepare texts and labels
texts = []
labels = []

# Set to keep track of added movie titles
seen_titles = set()

for label in df.columns:
    movies = df[label].dropna().tolist()
    for movie in movies:
        if movie not in seen_titles:
            texts.append(movie)
            labels.append(label)
            seen_titles.add(movie)

# Check the lengths to ensure they match
print(f"Total texts: {len(texts)}")
print(f"Total labels: {len(labels)}")

# Example print to verify
print(f"First text: {texts[0]}, Label: {labels[0]}")

Total texts: 419
Total labels: 419
First text: 10 Things I Hate About You, Label: romance


## 정민님이 하신건 419개뿐이라 이 파일로는 못함 >> 그래서 movie_details_no_scripted로 함

In [ ]:
import pandas as pd

# Load CSV files
movie_details_path = "/content/movie_details_no_scripted.csv"
scripts_preprocessed_path = "/content/scripts_preprocessed.csv"

df_movies = pd.read_csv(movie_details_path)
df_scripts = pd.read_csv(scripts_preprocessed_path)

# Ensure that title columns exist in both DataFrames
assert 'title' in df_movies.columns, "The movie details file must have a 'title' column"
assert 'title' in df_scripts.columns, "The scripts preprocessed file must have a 'title' column"

# Merge the DataFrames on the 'title' column (title열을 기준으로 하는 것, inner는 title 칼럼이 둘 다 존재할때만)
df_merged = pd.merge(df_scripts, df_movies[['title', 'genre']], on='title', how='inner')

# Rename the 'genre' column to 'label'
df_merged = df_merged.rename(columns={'genre': 'label'})

Labeling complete. Updated scripts saved to: /content/scripts_with_labels.csv


In [ ]:
len(df_merged)

1127

In [ ]:
len(df_scripts)

1123

병합하고 난 후 원본과 길이가 다름;;

In [ ]:
df_merged[-10:-1]

,title,script,label
1117,"X-Files: Fight the Future, The",x files production draft fight future chris ca...,"Action, Sci-Fi, Thriller"
1118,Year One,th beginning black infinite void center singul...,"Adventure, Comedy"
1119,Yes Man,yes man written based book yes man first draft...,"Comedy, Romance"
1120,X-Men Origins: Wolverine,xmen origins wolverine written skip woods ext ...,"Action, Fantasy, Sci-Fi, Thriller"
1121,xXx,xxx draft fade ext prague night hill city old ...,"Action, Adventure, Thriller"
1122,You Can Count On Me,count shooting draft fade intext car moving ni...,Drama
1123,Youth in Revolt,youth revolt written black comes sound deep he...,"Comedy, Drama, Romance"
1124,Zootopia,story byron jared bush jim reardon josie trini...,"Animation, Adventure, Comedy"
1125,Zero Dark Thirty,dark thirty written rd emerge hear actual reco...,"Drama, Thriller"


In [ ]:
df_scripts[-10:-1]

,title,script
1113,"X-Files: Fight the Future, The",x files production draft fight future chris ca...
1114,Year One,th beginning black infinite void center singul...
1115,Yes Man,yes man written based book yes man first draft...
1116,X-Men Origins: Wolverine,xmen origins wolverine written skip woods ext ...
1117,xXx,xxx draft fade ext prague night hill city old ...
1118,You Can Count On Me,count shooting draft fade intext car moving ni...
1119,Youth in Revolt,youth revolt written black comes sound deep he...
1120,Zootopia,story byron jared bush jim reardon josie trini...
1121,Zero Dark Thirty,dark thirty written rd emerge hear actual reco...


In [ ]:
# Check for duplicates in the 'title' column of df_movies
duplicates_in_movies = df_merged[df_movies['title'].duplicated(keep=False)]

# Check for duplicates in the 'title' column of df_scripts
duplicates_in_scripts = df_scripts[df_scripts['title'].duplicated(keep=False)]

# Print the duplicate titles
print("Duplicate titles in movie details:")
print(duplicates_in_movies)

print("\nDuplicate titles in scripts preprocessed:")
print(duplicates_in_scripts)

Duplicate titles in movie details:
            title                                             script  \
608     Labyrinth  l b r n h movie script scanned fade ext sky da...   
610   Lake Placid  lake placid written may full pink draft black ...   
813      Predator  b bif window top toplocationhreflocationhref b...   
815  Pretty Woman  p r e w n ext hollywood hills night view downt...   

                                   label  
608  Family, Fantasy, Adventure, Musical  
610     Action, Comedy, Horror, Thriller  
813     Action, Horror, Sci-Fi, Thriller  
815                      Comedy, Romance  

Duplicate titles in scripts preprocessed:
                               title  \
561            It Happened One Night   
562            It Happened One Night   
642                       Little Men   
643                       Little Men   
752  Nightmare Before Christmas, The   
754  Nightmare Before Christmas, The   

                                                script  
561  hap

<ipython-input-26-fd9377810c66>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  duplicates_in_movies = df_merged[df_movies['title'].duplicated(keep=False)]


## 중복되는 영화 하나만 남기기

In [ ]:
# Remove duplicate titles in df_movies
df_merged = df_merged.drop_duplicates(subset='title', keep='first')

# Remove duplicate titles in df_scripts
df_scripts = df_scripts.drop_duplicates(subset='title', keep='first')

In [ ]:
print(len(df_merged), len(df_scripts))

1120 1120


In [ ]:
# Save the merged DataFrame to a new CSV file
output_path = "/content/scripts_with_labels.csv"
df_merged.to_csv(output_path, index=False)

print("Labeling complete. Updated scripts saved to:", output_path)

Labeling complete. Updated scripts saved to: /content/scripts_with_labels.csv


In [ ]:
df_merged.head()

,title,script,label
0,12 and Holding,holding written fade ext neighborhood street m...,Drama
1,187,revised shooting draft educational purposes re...,Drama
2,1492: Conquest of Paradise,b bif window top toplocationhreflocationhref b...,"Adventure, Drama"
3,17 Again,written october ext fitch senior high school d...,"Comedy, Drama, Romance"
4,15 Minutes,b bif window top toplocationhreflocationhref b...,"Action, Crime, Thriller"


# 3. 일단 오타 고치지 않고 학습(fine-tuning) 시작

In [ ]:
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers import models, datasets
from torch.utils.data import DataLoader
from transformers import AdamW
import torch

# Step 1: Load the dataset
input_csv_path = "/content/scripts_with_labels.csv"
df = pd.read_csv(input_csv_path)

# Assuming the CSV has two columns: 'text' and 'label'
texts = df['script'].tolist()
labels = df['label'].tolist()

# Step 2: Split the dataset into training, validation, and test sets (70 15 15)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

# Step 3: Create InputExamples for Sentence Transformers (학습, 검증, 테스트 데이터셋을 모델에 입력할 수 있는 형식으로 변환; text/label)
train_examples = [InputExample(texts=[text], label=label) for text, label in zip(train_texts, train_labels)]
val_examples = [InputExample(texts=[text], label=label) for text, label in zip(val_texts, val_labels)]
test_examples = [InputExample(texts=[text], label=label) for text, label in zip(test_texts, test_labels)]

# Step 4: Load the Sentence Transformer model
model_name = 'sentence-transformers/all-MiniLM-L6-v2'  # SOTA
model = SentenceTransformer(model_name)

# Step 5: Define a DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=32)
test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=32)

# Step 6: Define the loss function
loss = losses.CosineSimilarityLoss(model)

# Step 7: Prepare the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


DataLoader에 InputExample 객체를 직접 사용하면 TypeError가 발생합니다. 따라서 Dataset 클래스를 정의하고 이를 통해 데이터를 DataLoader로 제공해야 합니다.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
import torch

# Custom Dataset class
class MovieDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return example.texts[0], example.label

# Step 1: Load the dataset
input_csv_path = "/content/scripts_with_labels.csv"
df = pd.read_csv(input_csv_path)

# Assuming the CSV has two columns: 'script' and 'label'
texts = df['script'].tolist()
labels = df['label'].tolist()

# Step 2: Split the dataset into training, validation, and test sets (70 15 15)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

# Step 3: Create InputExamples for Sentence Transformers
train_examples = [InputExample(texts=[text], label=label) for text, label in zip(train_texts, train_labels)]
val_examples = [InputExample(texts=[text], label=label) for text, label in zip(val_texts, val_labels)]
test_examples = [InputExample(texts=[text], label=label) for text, label in zip(test_texts, test_labels)]

print('Length of Train : ',len(train_examples)) # 784
print('Length of Valid : ',len(val_examples)) # 168
print('Length of Test : ',len(test_examples)) # 168

# Step 4: Load the Sentence Transformer model
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 5: Define a DataLoader using the custom Dataset
batch_size = 32
train_dataset = MovieDataset(train_examples)
val_dataset = MovieDataset(val_examples)
test_dataset = MovieDataset(test_examples)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Step 6: Define the loss function
loss = losses.CosineSimilarityLoss(model)

# Step 7: Prepare the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

Length of Train :  784
Length of Valid :  168
Length of Test :  168


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Step 8: Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for step, (inputs, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model.encode(inputs, convert_to_tensor=True)
        label_outputs = model.encode(labels, convert_to_tensor=True)
        loss_value = loss(outputs, label_outputs)
        loss_value.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs} completed")

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = model.encode(inputs, convert_to_tensor=True)
            label_outputs = model.encode(labels, convert_to_tensor=True)
            loss_value = loss(outputs, label_outputs)
            val_loss += loss_value.item()
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation Loss after epoch {epoch+1}: {avg_val_loss}")

# Step 9: Evaluate the model on the test set
model.eval()
test_loss = 0
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs = model.encode(inputs, convert_to_tensor=True)
        label_outputs = model.encode(labels, convert_to_tensor=True)
        loss_value = loss(outputs, label_outputs)
        test_loss += loss_value.item()
avg_test_loss = test_loss / len(test_dataloader)
print(f"Test Loss: {avg_test_loss}")

IndexError: too many indices for tensor of dimension 1

## 구글링해보니 mutil-classfication 이라 나온 문제라고 함

## + loss 높아서 이상해서 learning rate lr=1e-5로 낮추기

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
from transformers import AdamW

# Custom Dataset class
class MovieDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return example.texts[0], example.label

# Step 1: Load the dataset
input_csv_path = "/content/scripts_with_labels.csv"
df = pd.read_csv(input_csv_path)

# Assuming the CSV has two columns: 'script' and 'label'
texts = df['script'].tolist()
labels = df['label'].tolist()

# 레이블을 정수 인코딩으로 변환
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Step 2: 학습, 검증, 테스트 셋으로 데이터 분할 (70 15 15 비율)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, encoded_labels, test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

# Step 3: Sentence Transformers를 위한 InputExample 생성
train_examples = [InputExample(texts=[text], label=label) for text, label in zip(train_texts, train_labels)]
val_examples = [InputExample(texts=[text], label=label) for text, label in zip(val_texts, val_labels)]
test_examples = [InputExample(texts=[text], label=label) for text, label in zip(test_texts, test_labels)]

# Step 4: Sentence Transformer 모델 로드
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 5: 커스텀 Dataset을 사용하여 DataLoader 정의
batch_size = 32
train_dataset = MovieDataset(train_examples)
val_dataset = MovieDataset(val_examples)
test_dataset = MovieDataset(test_examples)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, collate_fn=lambda x: x)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size, collate_fn=lambda x: x)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, collate_fn=lambda x: x)

# Step 6: 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류 문제를 위한 손실 함수
optimizer = AdamW(model.parameters(), lr=1e-5)  # 옵티마이저 설정

# Step 7: Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0  # 총 손실 초기화

    for step, batch in enumerate(train_dataloader):
        texts, labels = zip(*batch)  # 배치에서 텍스트와 레이블 분리
        texts = list(texts)  # 텍스트 리스트로 변환
        labels = torch.tensor(labels, dtype=torch.long)  # 레이블 텐서로 변환 (정수_long 인코딩 했었음, loss에서 사용함)
        optimizer.zero_grad()  # 옵티마이저의 기울기 초기화

        # SentenceTransformer 모델의 encode 메서드는 기본적으로 텐서의 그래디언트를 계산하지 않도록 설정되어 있습니다.
        # 이를 해결하기 위해 모델을 학습 모드로 설정하고, 그래디언트를 계산할 수 있도록 합니다.
        with torch.set_grad_enabled(True):
            outputs = model.encode(texts, convert_to_tensor=True)  # 텍스트를 벡터로 인코딩
            outputs.requires_grad_(True)  # 출력 텐서의 기울기 계산 활성화 !!!
            loss_value = criterion(outputs, labels)  # 손실 계산
            loss_value.backward()  # 역전파를 통해 기울기 계산
            optimizer.step()  # 옵티마이저를 통해 가중치 업데이트
            total_loss += loss_value.item()  # 총 손실에 현재 손실 추가

    avg_train_loss = total_loss / len(train_dataloader)  # 평균 학습 손실 계산
    print(f"Epoch {epoch+1}/{num_epochs} - Average Training Loss: {avg_train_loss}")

    # Validation
    model.eval()  # 모델을 평가 모드로 설정
    val_loss = 0  # 검증 손실 초기화
    with torch.no_grad():  # 기울기 계산 비활성화
        for batch in val_dataloader:
            texts, labels = zip(*batch)  # 배치에서 텍스트와 레이블 분리
            texts = list(texts)  # 텍스트 리스트로 변환
            labels = torch.tensor(labels, dtype=torch.long)  # 레이블 텐서로 변환
            outputs = model.encode(texts, convert_to_tensor=True)  # 텍스트를 벡터로 인코딩
            loss_value = criterion(outputs, labels)  # 손실 계산
            val_loss += loss_value.item()  # 검증 손실에 현재 손실 추가
    avg_val_loss = val_loss / len(val_dataloader)  # 평균 검증 손실 계산
    print(f"Validation Loss after epoch {epoch+1}: {avg_val_loss}")

# Step 8: Evaluate the model on the test set
model.eval()  # 모델을 평가 모드로 설정
test_loss = 0  # 테스트 손실 초기화
with torch.no_grad():  # 기울기 계산 비활성화
    for batch in test_dataloader:
        texts, labels = zip(*batch)  # 배치에서 텍스트와 레이블 분리
        texts = list(texts)  # 텍스트 리스트로 변환
        labels = torch.tensor(labels, dtype=torch.long)  # 레이블 텐서로 변환
        outputs = model.encode(texts, convert_to_tensor=True)  # 텍스트를 벡터로 인코딩
        loss_value = criterion(outputs, labels)  # 손실 계산
        test_loss += loss_value.item()  # 테스트 손실에 현재 손실 추가
avg_test_loss = test_loss / len(test_dataloader)  # 평균 테스트 손실 계산
print(f"Test Loss: {avg_test_loss}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10 - Average Training Loss: 5.952108898162842
Validation Loss after epoch 1: 5.946971972783406
Epoch 2/10 - Average Training Loss: 5.952440700531006
Validation Loss after epoch 2: 5.946971972783406
Epoch 3/10 - Average Training Loss: 5.952239093780517
Validation Loss after epoch 3: 5.946971972783406


KeyboardInterrupt: 

# loss 인간적으로 너무 높아서 다시 다른 방법

In [ ]:
df['label'][3]

'Comedy, Drama, Romance'

# 1번 더 전처리 :label n개 = row n개에

In [ ]:
import pandas as pd

df = pd.read_csv("/content/scripts_with_labels.csv")

# 각 행을 여러 행으로 분할하는 함수
expanded_rows = []
for idx, row in df.iterrows():
    labels = row['label'].split(',')
    for label in labels:
        expanded_rows.append({'title': row['title'], 'script': row['script'], 'label': label})

# 새로운 데이터프레임 생성
expanded_df = pd.DataFrame(expanded_rows)
print(expanded_df)

                           title  \
0                 12 and Holding   
1                            187   
2     1492: Conquest of Paradise   
3     1492: Conquest of Paradise   
4                       17 Again   
...                          ...   
2928                    Zootopia   
2929            Zero Dark Thirty   
2930            Zero Dark Thirty   
2931                  Zerophilia   
2932                  Zerophilia   

                                                 script      label  
0     holding written fade ext neighborhood street m...      Drama  
1     revised shooting draft educational purposes re...      Drama  
2     b bif window top toplocationhreflocationhref b...  Adventure  
3     b bif window top toplocationhreflocationhref b...      Drama  
4     written october ext fitch senior high school d...     Comedy  
...                                                 ...        ...  
2928  story byron jared bush jim reardon josie trini...     Comedy  
2929  dark thir

In [ ]:
expanded_df.to_csv("/content/scripts_with_labels_each.csv", index=False)

In [ ]:
# Step 10: Save the fine-tuned model
# locally
output_model_path = "fine_tuned_model"
model.save(output_model_path)

# Save the model to Hugging Face Hub
# Specify your Hugging Face Hub model repository name
repo_name = "Uiji/movie-search-query-all-MiniLM-L6-v2"

# Create a repository on Hugging Face Hub
create_repo(repo_name, exist_ok=True)

# Push the model to Hugging Face Hub
model.push_to_hub(repo_name)

print("Fine-tuning completed and model saved to Hugging Face Hub.")